In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [15]:
teams_url = 'http://www.espn.com/nba/teams'
players_url = ''

In [16]:
html = urllib.request.urlopen(teams_url)
soup = BeautifulSoup(html, 'lxml')

In [17]:
links_store = str(soup.find_all('a'))

pattern = re.compile(r'href="/nba/team/_/name/\w*/\w*[-]?\w*')
matches = pattern.finditer(links_store)

teams = set()
for match in matches: 
    teams.add(match.group(0))

abbr_name = []
for link in teams:
    temp = link.split('/')
    abbr_name.append([temp[5], temp[6]])

abbr_name

[['atl', 'atlanta-hawks'],
 ['por', 'portland-trail'],
 ['mil', 'milwaukee-bucks'],
 ['tor', 'toronto-raptors'],
 ['okc', 'oklahoma-city'],
 ['hou', 'houston-rockets'],
 ['ind', 'indiana-pacers'],
 ['sa', 'san-antonio'],
 ['sac', 'sacramento-kings'],
 ['gs', 'golden-state'],
 ['bkn', 'brooklyn-nets'],
 ['chi', 'chicago-bulls'],
 ['den', 'denver-nuggets'],
 ['wsh', 'washington-wizards'],
 ['mem', 'memphis-grizzlies'],
 ['cle', 'cleveland-cavaliers'],
 ['mia', 'miami-heat'],
 ['orl', 'orlando-magic'],
 ['phi', 'philadelphia-76ers'],
 ['det', 'detroit-pistons'],
 ['dal', 'dallas-mavericks'],
 ['bos', 'boston-celtics'],
 ['lal', 'los-angeles'],
 ['utah', 'utah-jazz'],
 ['phx', 'phoenix-suns'],
 ['lac', 'la-clippers'],
 ['no', 'new-orleans'],
 ['min', 'minnesota-timberwolves'],
 ['cha', 'charlotte-hornets'],
 ['ny', 'new-york']]

In [18]:
home_split_format = 'https://www.espn.in/nba/team/stats/_/name/{team_abbr}/season/{season_no}/seasontype/{season_type}/split/{game_type}'
# codes for season type [2:Regular Season, 3:Post Season]
# codes for game_type [33: Home, 34:Away]
# data is available from 2002 - 2020

In [51]:
# # scrape the data for home games
# game_type = 33
shot_df = pd.DataFrame(columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', '2PM', '2PA', '2P%', 'SC-EFF', 'SH-EFF'])
year_list = range(2002, 2021)
for item in abbr_name:
    team_abbr = item[0]
    for year in year_list:
        season_no = year
        season_type = '2'
        game_types = ['33', '34']
        for game_type in game_types:
            time.sleep(3)
            test_split_link = 'https://www.espn.in/nba/team/stats/_/name/{team_abbr}/season/{season_no}/seasontype/{season_type}/split/{game_type}'.format(team_abbr=team_abbr, season_no=season_no, season_type=season_type, game_type=game_type)
            print(test_split_link)
            test_html = urllib.request.urlopen(test_split_link)
            test_soup = BeautifulSoup(test_html, 'lxml')
            players = []
            try:
                tables = test_soup.find_all('table')
                player_names = tables[2]
                rows = player_names.find_all('tr')
                # ignore first row
                for row in rows[1:]:
                    tds = row.find_all('td')[0]
                    spans = tds.find_all('span')[0]
                    try:
                        temp = [span.getText() for span in spans]
                        # remove trailing '*' after some of the position values
                        if len(temp) > 2:
                            temp = temp[:-1]
                        # removing leading spaces for position
                        temp[1] = temp[1].lstrip(' ')
                        players.append(temp)
                    except Exception as e:
                        pass

                all_rows = []
                tables = test_soup.find_all('table')
                player_stats = tables[3]
                rows = player_stats.find_all('tr')
                for row in rows[1:-1]:
                    tds = row.find_all('td')
                    vals = [td.getText() for td in tds]
                    all_rows.append(vals)

                concat_shot_stats = []
                for idx in range(len(players)):
                    try:
                        concat_shot_stats.append(players[idx] + [team_abbr] + [season_no] + [game_type] +  all_rows[idx])
                    except Exception as e:
                        pass
                temp_df = pd.DataFrame(concat_shot_stats,columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', '2PM', '2PA', '2P%', 'SC-EFF', 'SH-EFF'])
                shot_df = pd.concat([shot_df, temp_df], axis=0)
            except Exception as e:
                pass


https://www.espn.in/nba/team/stats/_/name/atl/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/atl/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/atl/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/atl/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/atl/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/atl/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/atl/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/mil/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mil/season/2016/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mil/season/2016/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mil/season/2017/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mil/season/2017/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mil/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mil/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mil/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mil/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mil/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mil/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/tor/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/hou/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2010/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/hou/season/2011/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2011/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/hou/season/2012/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2012/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/hou/season/2013/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2013/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/hou/season/2014/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2014/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/hou/season/2015/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/hou/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/sac/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/sac/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/sac/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/sac/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2008/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/sac/season/2008/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2009/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/sac/season/2009/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/sac/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/bkn/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bkn/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bkn/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bkn/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bkn/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bkn/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

...

https://www.espn.in/nba/team/stats/_/name/phi/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/bos/season/2015/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bos/season/2016/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2016/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bos/season/2017/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2017/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bos/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bos/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/bos/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/bos/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/phx/season/2009/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2009/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2010/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2011/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2011/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2012/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2012/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2013/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2013/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2014/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2014/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/min/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2008/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2008/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2009/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/ny/season/2017/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ny/season/2017/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ny/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ny/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ny/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ny/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ny/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ny/season/2020/seasontype/2/split/34


In [52]:
shot_df.head(40)

,PNAME,POS,TEAM,YEAR,TYPE,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,Shareef Abdur-Rahim,PF,atl,2002,33,8.1,17.1,47.4,0.4,1.1,39.0,5.2,6.4,81.6,7.7,16.0,48.0,1.279,0.49
1,Jason Terry,G,atl,2002,33,7.3,16.3,44.6,2.3,5.7,39.9,3.9,4.7,82.0,5.0,10.6,47.1,1.269,0.52
2,Theo Ratliff,C,atl,2002,33,5.0,8.0,62.5,0.0,0.0,0.0,2.0,6.0,33.3,5.0,8.0,62.5,1.500,0.63
3,Toni Kukoc,SF,atl,2002,33,4.1,9.1,45.3,1.1,3.0,36.7,2.0,2.7,76.3,3.0,6.1,49.5,1.248,0.51
4,Dion Glover,SG,atl,2002,33,4.5,9.8,46.3,0.8,2.3,36.8,1.3,1.6,78.0,3.7,7.5,49.2,1.143,0.51
5,Nazr Mohammed,C,atl,2002,33,4.3,9.2,47.0,0.0,0.0,0.0,2.0,3.2,61.7,4.3,9.2,47.0,1.156,0.47
6,DerMarr Johnson,SF,atl,2002,33,3.3,8.3,40.1,1.6,4.0,39.7,1.0,1.4,76.1,1.7,4.3,40.4,1.117,0.50
7,Ira Newble,SF,atl,2002,33,3.4,6.4,53.3,0.0,0.1,0.0,2.3,2.6,90.7,3.4,6.3,54.5,1.430,0.53
8,Jacque Vaughn,PG,atl,2002,33,2.7,5.0,53.2,0.3,0.6,52.0,1.0,1.3,79.2,2.3,4.4,53.3,1.332,0.56
9,Alan Henderson,PF,atl,2002,33,2.8,5.5,51.2,0.1,0.1,100.0,0.9,1.5,59.1,2.7,5.4,50.6,1.195,0.52


In [53]:
shot_df.to_csv('shots_home_away_stats.csv', index=False)